# Extracting and displaying the water level of Lake Zurich

In [59]:
import requests #Import requests library for doing the API calls

import plotly.offline as py #Import plotly (offline)
import plotly.graph_objs as go

import time #For the scheduler

### Test wheter accessing the API is possible

In [60]:
#State the API URL
base_url = 'https://tecdottir.metaodi.ch/measurements/station_name?sort=timestamp_cet%20desc&limit=100&offset=0'
base_url = base_url.replace("station_name", "mythenquai") #Replace the station name based on the two options: 'tiefenbrunnen' or 'mythenquai'


In [61]:
# Make a GET request to the API based on the following get_post function
def get_post():
    # GET request
    response = requests.get(base_url)

    # Check if the request was successful (status code 200)
    if response.status_code == 200:
        post = response.json()
        return post
    else:
        print('Error:', response.status_code)
        return None

In [62]:
#Inspect the file --> it consists of an array of values recorded at different time-steps for every 10 mins., where each value is a dictionary 
test = get_post()

In [63]:
test['result'][0]['values']['water_level']['value'] #Check the latest water level value

405.99

### Acquire the values for every 30 mins and plot them on a gauge chart with plotly

In [58]:
if __name__ == "__main__":
    time.sleep(60)  # initial waiting time: 1 minute
    while True:
        post = get_post()
        water_level = post['result'][0]['values']['water_level']['value'] #Get the water level value for every 30 mins.
        
        #Plot the gauge chart using plotly
        fig = go.Figure(go.Indicator(  
            mode = "gauge+number+delta",
            value = water_level,
            domain = {'x': [0, 1], 'y': [0, 1]},
            #title = {'x': 0, 'text': "Water level Lake of Zurich", 'font': {'size': 24}},
            delta = {'reference': 406.06, 'increasing': {'color': "RebeccaPurple"}},
            gauge = {
                'axis': {'range': [406, 407], 'tickwidth': 1, 'tickcolor': "darkblue"},
                'bar': {'color': "darkblue"},
                'bgcolor': "white",
                'borderwidth': 2,
                'bordercolor': "gray",
                'steps': [
                    {'range': [0, 406.25], 'color': 'white'},
                    {'range': [406.25, 406.4], 'color': 'yellow'},
                    {'range': [406.4, 406.6], 'color': 'orange'},
                    {'range': [406.6, 406.85], 'color': 'red'},
                    {'range': [406.85, 407], 'color': 'brown'}],
                'threshold': {
                    'line': {'color': "red", 'width': 4},
                    'thickness': 0.5,
                    'value': 406.06}}))

        fig.update_layout(title_text='Water level of Lake of Zurich from station Mythenquai', title_x=0.5, title_y=0.95)
        
        py.plot(fig)
        
        time.sleep(1800)

KeyboardInterrupt: 